In [1]:
import bz2
import nltk
import numpy as np
import json
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/leo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [92]:
train_file = bz2.BZ2File('../input_data/train.json.bz2')
#test_file = bz2.BZ2File('../input_data/test.json.bz2')
train_file = json.load(train_file)
#test_file = json.load(test_file)

In [93]:
idx_to_link = dict()

for i, sentence in enumerate(train_file):
    sentence, link = next(iter(sentence.items()))
    train_file[i] = sentence
    if ("_" in sentence):
        print(sentence)
    if (link is not None):
        idx_to_link[i] = link
        train_file[i] += "(" + str(i) + ")"

__NOTOC__   Bertrand Andrieu was a 
 in 1623, removing the dignity from the County Palatine of the Rhine   Coat_of_Arms_of_George_I_Louis,_Elector_of_Hanover_.svg|
 h, defined as:     : y \int_^ x\, h\, d\tau   : y_k \sum_^ x_\, h_i     The first form is the continuous-time form, which describes mechanical and analog electronic systems, for instance. The second equation is a discrete-time version used, for example, by digital filters implemented in software, so-called  digital signal processing . The impulse response h completely characterizes any linear time-invariant filter. The input x is said to be 
: Due to always running on as much air as possible and not depending on exact mixture of air and fuel, diesel engines have an air-fuel ratio leaner than stochiometric  !-- detonation from occurring after ignition and is more a fuel quality or combustion chamber design issue --  ref name Reif_2014_10 / ref name Hemmerlein_1991 /  ref name Diesel_1893_51 / ref name vB_2017_755 / ref name 

In [88]:
my_sentences = nltk.sent_tokenize(" ".join(train_file[:21]))
nltk.word_tokenize(my_sentences[0])


['Ciao', 'luca_io']

In [40]:
tmp_train_file = []
for sentence in train_file:
    #print(nltk.word_tokenize(sentence.keys()))
    sentence, link = next(iter(sentence.items()))
    sentence = nltk.word_tokenize(sentence)
    for word in sentence:
        tmp_train_file.append({word:link})
del train_file
train_file = tmp_train_file
del tmp_train_file

In [42]:

print(len(train_file))
corpora = " ".join([next(iter(token.keys())) for token in train_file])
print("(" in corpora)

6228632
False
